In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import cvxpy as cp
import gurobipy

In [ ]:
dh_s = 120*60
initial_storage_MW = 10000.0
initial_storage_MW = 0.0
max_discharges = 3.0

N = 12
Ts = []
for ii in range(0,288, int(288/N)):
    Ts.append(ii)

In [ ]:
supply_df = pd.read_csv('CAISO-supply-20210520.csv')
demand_df = pd.read_csv('CAISO-demand-20210510.csv')
renewables_df = pd.read_csv('CAISO-renewables-20210520.csv')

In [ ]:
natural_gas = supply_df.iloc[1].values[1:-1][Ts]
large_hydro = supply_df.iloc[2].values[1:-1][Ts]
imports = supply_df.iloc[3].values[1:-1][Ts]
batteries = supply_df.iloc[4].values[1:-1][Ts]
nuclear = supply_df.iloc[5].values[1:-1][Ts]
coal = supply_df.iloc[6].values[1:-1][Ts]
other = supply_df.iloc[7].values[1:-1][Ts]

solar = renewables_df.iloc[0].values[1:][Ts]
wind = renewables_df.iloc[1].values[1:][Ts]
geothermal = renewables_df.iloc[2].values[1:][Ts]
biomass = renewables_df.iloc[3].values[1:][Ts]
biogas = renewables_df.iloc[4].values[1:][Ts]
small_hydro = renewables_df.iloc[5].values[1:][Ts]

renewables = supply_df.iloc[0].values[1:-1][Ts]
non_renewables = natural_gas + large_hydro + imports + batteries + nuclear + coal + other

n_sources = 7
supply = np.zeros((n_sources, len(Ts)))
supply[0,:] = solar
supply[1] = wind
supply[2] = geothermal
supply[3] = biomass
supply[4] = biogas
supply[5] = small_hydro
supply[6] = non_renewables

In [ ]:
day_ahead_forecast = demand_df.iloc[0].values[1:-1][Ts]
hour_ahead_forecast = demand_df.iloc[1].values[1:-1][Ts]
# demand = demand_df.iloc[2].values[1:-1]

In [ ]:
y = cp.Variable(N, boolean=True)
M_ = 1e8

in_storage = cp.Variable(N+1)
store_in = cp.Variable(N)
store_out = cp.Variable(N)
buy = cp.Variable((n_sources,N))

In [ ]:
cons = []

cons += [in_storage >= 0.0]
cons += [store_in >= 0.0]
cons += [store_out >= 0.0]
cons += [buy >= 0.0]

# Storage initialization constraint
cons += [in_storage[0] == initial_storage_MW]

for ii in range(N):
    # Available power constraint
    for jj in range(n_sources):
        cons += [buy[jj,ii] <= np.maximum(supply[jj,ii], 0.0)]

    # Storage conservation of energy
    cons += [in_storage[ii+1] == in_storage[ii] + store_in[ii] - store_out[ii]]

    # Sufficiency constraint
    cons += [sum(buy[:,ii]) - store_in[ii] + store_out[ii] >= hour_ahead_forecast[ii]]

    # Store-in constraint
    cons += [store_in[ii] <= sum(buy[:,ii])]

    # Store-out constraint
    cons += [store_out[ii] <= M_*y[ii]]

for ii in range(0,N,3):
    min_idx, max_idx = ii, np.minimum(ii+3,N-1)
    cons += [cp.sum(y[min_idx:max_idx]) <= max_discharges]

In [ ]:
total_cost = cp.sum(buy)
prob = cp.Problem(cp.Minimize(total_cost), cons)
prob.solve(solver=cp.GUROBI)

In [ ]:
plt.plot(Ts+[Ts[-1]+24], in_storage.value, label="In Storage [MW]")
plt.plot(Ts, store_in.value, label="Store In [MW]")
plt.plot(Ts, store_out.value, label="Store Out [MW]")
plt.legend(loc="best")